In [1]:
import time
import subprocess
import matplotlib.pyplot as plt
import psutil
import os
import re
import threading
import csv


In [2]:
programs = ["./build/fuzzer","./build/vmfuzzer"]

In [3]:
directory = './grammars'
files = os.listdir(directory)
print(files)


['math.json', 'recursive.json', 'simple_grammar.json', 'css.json', 'html.json']


In [4]:
depth = [1,2,4,8,16,32,64,128,256,512,1024,2048]

In [5]:
#process = subprocess.Popen([programs[0],"-depth","2048", "-path", "./grammars/html.json", "-o", "html.c"],stderr=subprocess.PIPE,stdout=subprocess.PIPE)

In [6]:
throughput_rates = []
def monitor_process(process, timeout):
    time.sleep(timeout)
    process.kill()

def read_output(process):
    global throughput_rates
    while True:
        output = process.stdout.readline()
        if process.poll() is not None and output == b'':
            break
        if output:
            match = re.search(r'Throughput Rate:\s*([\d.]+)\s*MB/s', output.decode('utf-8'))
            if match:
                throughput_rate = float(match.group(1))
                throughput_rates.append(throughput_rate)

result = {}
timeout = 600  # 10 minutes
for i in range(len(programs)):
    program_name = programs[i]
    result[program_name] = {}
    for j in range(len(files)):
        file_name = files[j]
        result[program_name][file_name] = {}
        for k in range(len(depth)):
            print("{} {} {}:start".format(program_name, file_name, depth[k]))
            depth_value = depth[k]
            throughput_rates = []
            process = subprocess.Popen(
                [programs[i], "-depth", str(depth[k]), "-path", "./grammars/" + files[j], "-o", "output.c"],
                stderr=subprocess.PIPE,
                stdout=subprocess.PIPE
            )
            monitor_thread = threading.Thread(target=monitor_process, args=(process, timeout))
            monitor_thread.start()
            read_output(process)
            monitor_thread.join()
            if throughput_rates:
                avg_throughput_rate = sum(throughput_rates) / len(throughput_rates)
            else:
                avg_throughput_rate = 0
            result[program_name][file_name][depth_value] = avg_throughput_rate
            print(program_name, file_name, depth_value, avg_throughput_rate)

print(result)

KeyboardInterrupt: 

In [ ]:
with open('results.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Program', 'File', 'Depth', 'Average Throughput Rate (MB/s)'])
    for program_name, files_dict in result.items():
        for file_name, depths_dict in files_dict.items():
            for depth_value, avg_throughput_rate in depths_dict.items():
                writer.writerow([program_name, file_name, depth_value, avg_throughput_rate])

print("Results have been written to results.csv")